In [2]:
# --- Imports & Path check ---
from pathlib import Path
import pandas as pd
import duckdb

DATA_PATH = Path("../data/clean_events.csv")
assert DATA_PATH.exists(), f"File not found: {DATA_PATH.resolve()}"
DATA_PATH.resolve()


WindowsPath('D:/project/ABTest/data/clean_events.csv')

In [3]:
# --- DuckDB connect & create view ---
con = duckdb.connect(database=":memory:")

con.execute(f"""
CREATE OR REPLACE VIEW events AS
SELECT *
FROM read_csv_auto('{DATA_PATH.as_posix()}');
""")

# quick sanity check
con.execute("SELECT COUNT(*) AS n_rows FROM events").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,n_rows
0,109950731


In [4]:
# --- filter valid events ---
con.execute("""
CREATE OR REPLACE VIEW funnel_events AS
SELECT user_id, event_time, event_type
FROM events
WHERE event_type IN ('view', 'cart', 'purchase')
  AND user_id IS NOT NULL
  AND event_time IS NOT NULL;
""")

con.execute("""
SELECT event_type, COUNT(*) AS cnt
FROM funnel_events
GROUP BY 1
ORDER BY cnt DESC;
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,event_type,cnt
0,view,104335509
1,cart,3955434
2,purchase,1659788


In [5]:
# --- first_seen_time per user ---
con.execute("""
CREATE OR REPLACE TABLE user_first_seen AS
SELECT
  user_id,
  MIN(event_time) AS first_seen_time
FROM funnel_events
GROUP BY user_id;
""")

con.execute("SELECT COUNT(*) AS n_users FROM user_first_seen;").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,n_users
0,5316649


In [6]:
# --- define new/returning users by first_seen window ---
NEW_USER_WINDOW_DAYS = 7

con.execute(f"""
CREATE OR REPLACE TABLE user_segment AS
SELECT
  user_id,
  first_seen_time,
  CASE
    WHEN first_seen_time >= (SELECT MAX(first_seen_time) FROM user_first_seen) - INTERVAL '{NEW_USER_WINDOW_DAYS} days'
      THEN 'new'
    ELSE 'returning'
  END AS user_type
FROM user_first_seen;
""")

con.execute("""
SELECT user_type, COUNT(*) AS users
FROM user_segment
GROUP BY 1
ORDER BY users DESC;
""").df()


,user_type,users
0,returning,4877171
1,new,439478


In [7]:
# --- strict-order timestamps per user ---
# rule:
# view_time = earliest view
# cart_after_view_time = earliest cart after view_time
# purchase_after_cart_time = earliest purchase after cart_after_view_time

con.execute("""
CREATE OR REPLACE TABLE user_funnel_strict AS
WITH base AS (
  SELECT user_id, event_time, event_type
  FROM funnel_events
),

view_t AS (
  SELECT user_id, MIN(event_time) AS view_time
  FROM base
  WHERE event_type = 'view'
  GROUP BY user_id
),

cart_t AS (
  SELECT
    b.user_id,
    MIN(b.event_time) AS cart_time
  FROM base b
  JOIN view_t v
    ON b.user_id = v.user_id
  WHERE b.event_type = 'cart'
    AND b.event_time > v.view_time
  GROUP BY b.user_id
),

purchase_t AS (
  SELECT
    b.user_id,
    MIN(b.event_time) AS purchase_time
  FROM base b
  JOIN cart_t c
    ON b.user_id = c.user_id
  WHERE b.event_type = 'purchase'
    AND b.event_time > c.cart_time
  GROUP BY b.user_id
)

SELECT
  v.user_id,
  v.view_time,
  c.cart_time,
  p.purchase_time
FROM view_t v
LEFT JOIN cart_t c ON v.user_id = c.user_id
LEFT JOIN purchase_t p ON v.user_id = p.user_id;
""")

con.execute("""
SELECT
  COUNT(*) AS view_users,
  SUM(cart_time IS NOT NULL) AS view_to_cart_users,
  SUM(purchase_time IS NOT NULL) AS view_to_cart_to_purchase_users
FROM user_funnel_strict;
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,view_users,view_to_cart_users,view_to_cart_to_purchase_users
0,5316128,1052597.0,539119.0


In [8]:
# --- funnel by segment (strict order) ---
con.execute("""
CREATE OR REPLACE VIEW funnel_with_segment AS
SELECT
  s.user_type,
  f.user_id,
  f.view_time,
  f.cart_time,
  f.purchase_time
FROM user_funnel_strict f
JOIN user_segment s
  ON f.user_id = s.user_id;
""")

seg_funnel = con.execute("""
SELECT
  user_type,
  COUNT(*) AS view_users,
  SUM(cart_time IS NOT NULL) AS view_to_cart_users,
  SUM(purchase_time IS NOT NULL) AS view_to_cart_to_purchase_users,
  SUM(cart_time IS NOT NULL) * 1.0 / COUNT(*) AS view_to_cart_rate,
  SUM(purchase_time IS NOT NULL) * 1.0 / COUNT(*) AS view_to_cart_to_purchase_rate,
  CASE
    WHEN SUM(cart_time IS NOT NULL) = 0 THEN NULL
    ELSE SUM(purchase_time IS NOT NULL) * 1.0 / SUM(cart_time IS NOT NULL)
  END AS cart_to_purchase_rate
FROM funnel_with_segment
GROUP BY 1
ORDER BY view_users DESC;
""").df()

seg_funnel


,user_type,view_users,view_to_cart_users,view_to_cart_to_purchase_users,view_to_cart_rate,view_to_cart_to_purchase_rate,cart_to_purchase_rate
0,returning,4876696,1001021.0,514223.0,0.205266,0.105445,0.513699
1,new,439432,51576.0,24896.0,0.117370,0.056655,0.482705


In [9]:
# --- format output table ---
out = seg_funnel.copy()

for c in ["view_to_cart_rate", "view_to_cart_to_purchase_rate", "cart_to_purchase_rate"]:
    out[c] = (out[c] * 100).round(2)

out.rename(columns={
    "view_to_cart_rate": "view→cart (%)",
    "view_to_cart_to_purchase_rate": "view→cart→purchase (%)",
    "cart_to_purchase_rate": "cart→purchase (%)"
}, inplace=True)

out


,user_type,view_users,view_to_cart_users,view_to_cart_to_purchase_users,view→cart (%),view→cart→purchase (%),cart→purchase (%)
0,returning,4876696,1001021.0,514223.0,20.53,10.54,51.37
1,new,439432,51576.0,24896.0,11.74,5.67,48.27


## 4.3 Responsibility Segment & A/B Targeting Decision

### Segmentation Rule
Users are segmented into **new vs returning** based on their **first_seen_time** (earliest event_time observed in the funnel event set).
We define **new users** as those whose first_seen_time falls within the **last 7 days** of the observation window; all others are **returning**.

### Funnel Definition (Strict Order)
We use a strict-order funnel with timestamp constraints:
**view → cart → purchase**, requiring:
- cart_time > view_time
- purchase_time > cart_time

### Key Results (User-level)
- Returning users: view→cart = **20.53%**, view→cart→purchase = **10.54%**, cart→purchase = **51.37%**
- New users: view→cart = **11.74%**, view→cart→purchase = **5.67%**, cart→purchase = **48.27%**

### Conclusion: Responsibility Segment
The largest drop-off occurs at the **early-stage transition (view→cart)**, and the gap is driven primarily by **new users**.
Therefore, **new users are identified as the responsibility segment**, and **view→cart** is the primary optimization lever.

### A/B Experiment Implication
We will design the A/B intervention to target **new users at the view stage** (e.g., improving product-page trust/value signals or first-time incentives) with:
- **Primary KPI:** new-user view→cart rate
- **Secondary KPI:** new-user view→cart→purchase rate
- **Guardrails:** cart→purchase rate (and revenue-related metrics if available)


In [12]:
# --- Cell 10: Save Phase 4 outputs (Corrected Path Version) ---
from pathlib import Path
import pandas as pd
import pyarrow  # 显式依赖

# -------------------------------------------------------
# 0) Hard checks
# -------------------------------------------------------
assert "con" in globals(), "DuckDB connection `con` not found."
assert "out" in globals(), "`out` (segmented funnel summary) not found."

# -------------------------------------------------------
# 1) Define PROJECT-level data directory (IMPORTANT)
# notebooks/ -> ../data/
# -------------------------------------------------------
DATA_DIR = Path("../data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# -------------------------------------------------------
# Part 1: Save CSV outputs
# -------------------------------------------------------
summary_path = DATA_DIR / "segmented_funnel_summary.csv"
out.to_csv(summary_path, index=False)
print(f"Saved Summary: {summary_path}")

user_seg_df = con.execute("SELECT * FROM user_segment").df()
seg_path = DATA_DIR / "user_segment.csv"
user_seg_df.to_csv(seg_path, index=False)
print(f"Saved Segment Map: {seg_path} ({len(user_seg_df):,} rows)")

# -------------------------------------------------------
# Part 2: Save Parquet (Strict Funnel User-level Table)
# -------------------------------------------------------
tbl_check = con.execute("""
SELECT COUNT(*) AS cnt
FROM information_schema.tables
WHERE table_name = 'user_funnel_strict'
""").df()["cnt"].iloc[0]
assert tbl_check == 1, "`user_funnel_strict` table not found."

user_funnel_df = con.execute("SELECT * FROM user_funnel_strict").df()
parquet_path = DATA_DIR / "user_funnel_strict.parquet"
user_funnel_df.to_parquet(parquet_path, index=False)

# Read-back verification
chk_df = pd.read_parquet(parquet_path)
assert len(chk_df) == len(user_funnel_df), "Parquet row mismatch after save."

print(f"Saved Parquet Table: {parquet_path} ({len(user_funnel_df):,} rows)")
print("-" * 40)
print("Phase 4 outputs saved to PROJECT data/ successfully.")


Saved Summary: ..\data\segmented_funnel_summary.csv
Saved Segment Map: ..\data\user_segment.csv (5,316,649 rows)
Saved Parquet Table: ..\data\user_funnel_strict.parquet (5,316,128 rows)
----------------------------------------
Phase 4 outputs saved to PROJECT data/ successfully.
